# Bias Correction & Spacial Disaggregation Downscaling

The purpose of this notebook is to work as utility, taking low resolution climate model data and scaling it to high resolution while correcting for bias by comparison with historical observation data.

This notebook needs three things to run:
```
INPUT:
    1. Historical low obs data 
    2. Historical low model data
    3. low res model data of interest
OUTPUT:
    1. High-res version of the data from #3 above
```

In [1]:
import numpy as np
import xarray as xr
import bcsd

# Obs data
You're going to need an observation dataset with the same grid as your training model dataset over the training period.

[You can find a good source of data here](https://esgf.nccs.nasa.gov/esgdoc/NEX-GDDP_Tech_Note_v0.pdf)


# Model data
You'll need a model dataset that overlaps in time with the obs dataset. It doesnt have to be a perfect 1:1 match, but the more intersection there is the better the results will be.


In [17]:
OBS_DATA_PATH = "../BCSD-data/princeton/prec_monthly_1948-2016.nc"
MODEL_DATA_PATH = "../BCSD-data/GFDL/pr_day_GFDL-CM4_historical_r1i1p1f1_gr1_19500101-19691231.nc"

In [18]:
obs_data = xr.open_dataset(OBS_DATA_PATH)
print(obs_data)

<xarray.Dataset>
Dimensions:  (time: 828, lon: 1440, lat: 600)
Coordinates:
  * time     (time) datetime64[ns] 1948-01-01 1948-02-01 ... 2016-12-01
  * lon      (lon) float32 0.125 0.375 0.625 0.875 ... 359.1 359.4 359.6 359.9
  * lat      (lat) float32 -59.88 -59.62 -59.38 -59.12 ... 89.38 89.62 89.88
Data variables:
    data     (time, lat, lon) float32 ...
Attributes:
    CDI:          Climate Data Interface version 1.9.3 (http://mpimet.mpg.de/...
    Conventions:  CF-1.6
    history:      Mon Jan 11 11:29:20 2021: cdo -s -f nc4 -z zip import_binar...
    CDO:          Climate Data Operators version 1.9.3 (http://mpimet.mpg.de/...


/home/baldwin32/anaconda3/envs/bcsd/lib/python3.10/site-packages/xarray/coding/times.py:144: SerializationWarning: Ambiguous reference date string: 1-1-1 00:00:00. The first value is assumed to be the year hence will be padded with zeros to remove the ambiguity (the padded reference date string is: 0001-1-1 00:00:00). To remove this message, remove the ambiguity by padding your reference date strings with zeros.
  warnings.warn(warning_msg, SerializationWarning)


In [19]:
model_data = xr.open_dataset(MODEL_DATA_PATH)
print(model_data)

<xarray.Dataset>
Dimensions:    (lat: 180, bnds: 2, lon: 288, time: 7300)
Coordinates:
  * lat        (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * lon        (lon) float64 0.625 1.875 3.125 4.375 ... 355.6 356.9 358.1 359.4
  * time       (time) object 1950-01-01 12:00:00 ... 1969-12-31 12:00:00
Dimensions without coordinates: bnds
Data variables:
    lat_bnds   (lat, bnds) float64 ...
    lon_bnds   (lon, bnds) float64 ...
    pr         (time, lat, lon) float32 ...
    time_bnds  (time, bnds) object ...
Attributes: (12/46)
    external_variables:     areacella
    history:                File was processed by fremetar (GFDL analog of CM...
    table_id:               day
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   0.0
    ...                     ...
    variable_id:            pr
    variant_info:           N/A
    references:             see further_info_url attribute
    variant_label:          r1i1p1f1
    

In [23]:
model_time = model_data.indexes['time'].to_datetimeindex().values

/p/user_pub/e3sm/baldwin32/tmp/ipykernel_244404/556321923.py:1: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  model_time = model_data.indexes['time'].to_datetimeindex().values


In [25]:
d1 = xr.where(obs_data.time >= model_time[0], obs_data, None)
d2 = xr.where(obs_data.time <= model_time[-1], d1, None)
intersection = d2.dropna(dim='time')
del d1
del d2

In [26]:
intersection

<xarray.Dataset>
Dimensions:  (time: 239, lat: 600, lon: 1440)
Coordinates:
  * time     (time) datetime64[ns] 1950-02-01 1950-03-01 ... 1969-12-01
  * lon      (lon) float32 0.125 0.375 0.625 0.875 ... 359.1 359.4 359.6 359.9
  * lat      (lat) float32 -59.88 -59.62 -59.38 -59.12 ... 89.38 89.62 89.88
Data variables:
    data     (time, lat, lon) object 38.889007568359375 ... 20.529815673828125

In [15]:
intersection = d2.dropna(dim='time')

In [16]:
intersection

<xarray.Dataset>
Dimensions:  (time: 239, lat: 600, lon: 1440)
Coordinates:
  * time     (time) datetime64[ns] 1950-02-01 1950-03-01 ... 1969-12-01
  * lon      (lon) float32 0.125 0.375 0.625 0.875 ... 359.1 359.4 359.6 359.9
  * lat      (lat) float32 -59.88 -59.62 -59.38 -59.12 ... 89.38 89.62 89.88
Data variables:
    data     (time, lat, lon) object 38.889007568359375 ... 20.529815673828125